In [3]:
from importlib import reload
import book.pagerank_test as pt
import book.pagerank as pr
from book.mat import *
from book.vec import *

In [86]:
links = pr.read_data()

Reading word meta-index
Reading titles
Reading link structure
..................................................................................
Done


In [7]:
def find_num_links(L):
    result = Vec(L.D[1], {})
    for k, v in L.f.items():
        r, c = k
        result[c] += 1
    return result

num_links = find_num_links(links)

In [33]:
def make_Markov(L):
    num_links = find_num_links(L)
    for k, v in L.f.items():
        r, c = k
        L[k] = v / num_links[c]
    return L

In [40]:
reload(pt)
print(make_Markov(pt.small_links))


       1 2 3   4     5   6
     ---------------------
 1  |  1 0 0 0.5     0   0
 2  |  0 0 1 0.5 0.333 0.5
 3  |  0 1 0   0     0   0
 4  |  0 0 0   0 0.333   0
 5  |  0 0 0   0     0 0.5
 6  |  0 0 0   0 0.333   0



In [136]:
import math

def power_method(A, k, x_0=None, verbose=True):
    r1 = 0.85
    r2 = 0.15
    shape = (len(A.D[0]), len(A.D[1]))
    print(f"A.shape = {shape}")
    x = x_0 or Vec(A.D[1], {i:1 for i in A.D[1]})
    # A2_r = Vec(A.D[1], {i:1.0/shape[0] for i in A.D[1]})
    # Implicit computes Ax = A_1x + A_2x, leverage A2 is uniform
    for i in range(k):
        A2_r_x = 1.0/shape[0] * sum(x.f.values())
        x = r1 * A * x
        for j in x.f.keys(): 
            x[j] += r2 * A2_r_x
        x /= math.sqrt(x*x)
        if verbose:
            print(f"Iteration {i} finished.")
    return x

reload(pt)
make_Markov(pt.small_links)
A1 = pt.small_links # in-place update

x_t = power_method(A1, 50, None, False)
x_t

A.shape = (6, 6)


Vec({1, 2, 3, 4, 5, 6},{1: 0.5220380979609003, 2: 0.6181468460343382, 3: 0.5738401443097172, 4: 0.07052093604033213, 5: 0.07830571469413503, 6: 0.07052093604033213})

In [104]:
pr.find_word("jordan")[:10]

['alabama',
 'altruism',
 'asphalt',
 'arabic language',
 'attila the hun',
 'alexander graham bell',
 'albert speer',
 'aaron',
 'april 6',
 'april 12']

In [105]:
global_find_word = pr.find_word
def wikigoogle(w, k, p):
    related = global_find_word(w)
    # Why sort on eigen values, what's the meaning of eigen values magnitude?
    related.sort(key= lambda x:p[x], reverse=True)
    return related[:k]

In [137]:
p = power_method(links, 10, p)

A.shape = (825081, 825081)
Iteration 0 finished.
Iteration 1 finished.
Iteration 2 finished.
Iteration 3 finished.
Iteration 4 finished.
Iteration 5 finished.
Iteration 6 finished.
Iteration 7 finished.
Iteration 8 finished.
Iteration 9 finished.


In [133]:
wikigoogle("tiger", 100, p)
# global_find_word("matrix")[:10]

['france',
 '2006',
 'england',
 'india',
 '2001',
 '1945',
 '1944',
 '1975',
 '1998',
 '1979',
 '1948',
 '1993',
 '1985',
 '1996',
 '1997',
 '1937',
 '1984',
 '1966',
 '1962',
 '1936',
 '1989',
 '1990',
 '1922',
 '1912',
 '1959',
 'sweden',
 'ireland',
 'april 1',
 'november 11',
 'major league baseball',
 'february 27',
 'february 3',
 'march 16',
 'july 5',
 'july 23',
 'nato',
 'april 21',
 'asia',
 'serbia',
 'april 20',
 'december 30',
 'may 3',
 'august 14',
 'august 31',
 'december 20',
 'south korea',
 'supreme court of the united states',
 'indonesia',
 'january 25',
 'october 3',
 'september 27',
 'hong kong',
 'december 14',
 'december 7',
 'estonia',
 'april 13',
 'republic of ireland',
 'taiwan',
 'malaysia',
 'singapore',
 'republic of china',
 'quebec',
 'sri lanka',
 'sydney',
 'beijing',
 'bangladesh',
 'national football league',
 'dublin',
 'red army',
 'world series',
 'kenya',
 'dominican republic',
 'south carolina',
 'michigan',
 'baghdad',
 'new york yankees',


In [121]:
# Task 12.12.7
def power_method_biased(A, k, r_bias_label=None, verbose=True):
    r1 = 0.55
    r2 = 0.15
    r3 = 0.3
    shape = (len(A.D[0]), len(A.D[1]))
    x = Vec(A.D[1], {i:1 for i in A.D[1]})
    # Implicit computes Ax = A_1x + A_2x + A_3x, A_3 only have value 
    for i in range(k):
        A2_r_x = 1.0/shape[0] * sum(x.f.values())
        A3_r_x = sum(x.f.values())
        x = r1 * A * x
        for j in x.f.keys(): 
            x[j] += r2 * A2_r_x
            if r_bias_label == j:
                x[j] += r3 * A3_r_x
        x /= math.sqrt(x*x)
        if verbose:
            print(f"Iteration {i} finished.")
    return x

x_t = power_method_biased(A1, 50, 5, False)
x_t

Vec({1, 2, 3, 4, 5, 6},{1: 0.23534104242732637, 2: 0.4657101620939841, 3: 0.30963216850927805, 4: 0.19058868994445838, 5: 0.7478024213830594, 6: 0.19058868994445838})

In [ ]:
# wikigoogle2, use intersection on related article first